In [16]:
import wandb
import matplotlib.pyplot as plt
import scipy
import numpy as np
import json
import subprocess

plt.rcParams["axes.grid"] = False

sys.path.append('..')

from enums.evaluation_type import EvaluationType
from utils.dict_utils import update_dictionaries

In [2]:
def set_system_arguments():
    sys.argv = [
        "--device cuda",
        "--data-folder", "..\\data",
        "--checkpoint-folder", "..\\results",
        "--epochs", "100000",
        "--eval-freq", "30",
        "--seed", "13",
        "--configuration", "rnn-simple",
        "--learning-rate", "1e-2",
        "--metric-types", "f1-score", "precision", "recall",
        "--challenge", "named-entity-recognition",
        "--batch-size", "1",
        "--resume-training",

        "--no-attention",

        "--fine-tune-learning-rate", "1e-4",
        "--pretrained-model-size", "768",
        "--pretrained-max-length", "512",
        "--fasttext-model-size", "300",

        "--bidirectional-rnn",
        "--number-of-layers", "1",

        "--replace-all-numbers",

        "--merge-subwords",
        "--evaluate",

        # changeable

#         # "--checkpoint-name", "french-all--ft-bert-pretr-ce16-ch32-h256-e256-l1-bi-d0.80.0001-nonew-spl-ms",


#         "--learn-character-embeddings",
#         "--character-embeddings-size", "16",
#         "--character-hidden-size", "32",
        
#         "--entity-tag-types", "literal-fine", "literal-coarse", "metonymic-fine", "metonymic-coarse", "component", "nested",
#         "--split-type", "multi-segment",

#         # "--learn-new-embeddings",

#         "--hidden-dimension", "256",
#         "--embeddings-size", "256",
#         "--include-pretrained-model",
#         "--include-fasttext-model",
        
#         # "--fine-tune-pretrained",
#         # "--fine-tune-after-convergence",
        

    ]

In [ ]:
language_args = {
    'french': [
        "--language", "french",
        "--fasttext-model", "fr-model-skipgram-300minc20-ws5-maxn-6.bin",
        "--pretrained-weights", "bert-base-multilingual-cased"
    ]
    'german': [
        "--language", "german",
        "--fasttext-model", "de-model-skipgram-300-minc20-ws5-maxn-6.bin",
        "--pretrained-weights", "bert-base-german-cased"
    ]
    'english': [
        "--language", "english",
        "--fasttext-model", "en-model-skipgram-300-minc5-ws5-maxn-6.bin",
        "--pretrained-weights", "bert-base-cased"
    ]
}

specific_args = {
    'none-no-char': {
        'base_config': 'all--bert-h512-e64-l1-bi-d0.80.0001-spl-ms',
        'args': [
            "--entity-tag-types", "literal-fine", "literal-coarse", "metonymic-fine", "metonymic-coarse", "component", "nested",
            "--split-type", "multi-segment",

            "--learn-new-embeddings",

            "--hidden-dimension", "512",
            "--embeddings-size", "64"
        ]
    },
    'none': {
        'base_config': 'all--bert-ce16-ch32-h512-e64-l1-bi-d0.80.0001-spl-ms',
        'args': [
            "--entity-tag-types", "literal-fine", "literal-coarse", "metonymic-fine", "metonymic-coarse", "component", "nested",
            "--split-type", "multi-segment",

            "--learn-new-embeddings",

            "--hidden-dimension", "512",
            "--embeddings-size", "64",

            "--learn-character-embeddings",
            "--character-embeddings-size", "16",
            "--character-hidden-size", "32"
        ]
    },
    'fast-text': {
        'base_config': 'all--ft-bert-ce16-ch32-h512-e64-l1-bi-d0.80.0001-spl-ms',
        'args': [
            "--entity-tag-types", "literal-fine", "literal-coarse", "metonymic-fine", "metonymic-coarse", "component", "nested",
            "--split-type", "multi-segment",

            "--learn-new-embeddings",

            "--hidden-dimension", "512",
            "--embeddings-size", "64",

            "--learn-character-embeddings",
            "--character-embeddings-size", "16",
            "--character-hidden-size", "32",
            
#             "--include-pretrained-model",
            "--include-fasttext-model",

            # "--fine-tune-pretrained",
            # "--fine-tune-after-convergence",
        ]
    },
    'both': {
        'base_config': 'all--ft-bert-pretr-ce16-ch32-h256-e256-l1-bi-d0.80.0001-nonew-spl-ms',
        'args': [
            "--entity-tag-types", "literal-fine", "literal-coarse", "metonymic-fine", "metonymic-coarse", "component", "nested",
            "--split-type", "multi-segment",

            "--learn-new-embeddings",

            "--hidden-dimension", "512",
            "--embeddings-size", "64",

            "--learn-character-embeddings",
            "--character-embeddings-size", "16",
            "--character-hidden-size", "32",
            
            "--include-pretrained-model",
            "--include-fasttext-model",

            # "--fine-tune-pretrained",
            # "--fine-tune-after-convergence",
        ]
    },
    'bert': {
        'base_config': 'all--ft-bert-pretr-ce16-ch32-h256-e256-l1-bi-d0.80.0001-nonew-spl-ms',
        'args': [
            "--entity-tag-types", "literal-fine", "literal-coarse", "metonymic-fine", "metonymic-coarse", "component", "nested",
            "--split-type", "multi-segment",

            "--learn-new-embeddings",

            "--hidden-dimension", "256",
            "--embeddings-size", "64",

            "--learn-character-embeddings",
            "--character-embeddings-size", "16",
            "--character-hidden-size", "32",
            
            "--include-pretrained-model",
#             "--include-fasttext-model",

            # "--fine-tune-pretrained",
            # "--fine-tune-after-convergence",
        ]
    },
}

In [3]:
# Configure container:
from dependency_injection.ioc_container import IocContainer

container = IocContainer()

In [4]:
dataloader_service = container.dataloader_service()
model = container.model()
file_service = container.file_service()
evaluation_service = container.evaluation_service()

In [5]:
dataloader = dataloader_service.get_test_dataloader()
checkpoint_name = "BEST_french-all--ft-bert-pretr-ce16-ch32-h256-e256-l1-bi-d0.80.0001-nonew-spl-ms"
checkpoints_path = file_service.get_checkpoints_path()
model.load(checkpoints_path, 'BEST', checkpoint_name=checkpoint_name)
model.eval()
model.to('cuda')

Loaded 238 items for 'RunType.Test' set
Loaded BEST_french-all--ft-bert-pretr-ce16-ch32-h256-e256-l1-bi-d0.80.0001-nonew-spl-ms


NERPredictor(
  (rnn_encoder): RNNEncoder(
    (_embedding_layer): EmbeddingLayer(
      (_pretrained_layer): PretrainedRepresentationsLayer(
        (_pretrained_model): BertModel(
          (embeddings): BertEmbeddings(
            (word_embeddings): Embedding(119547, 768, padding_idx=0)
            (position_embeddings): Embedding(512, 768)
            (token_type_embeddings): Embedding(2, 768)
            (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (encoder): BertEncoder(
            (layer): ModuleList(
              (0): BertLayer(
                (attention): BertAttention(
                  (self): BertSelfAttention(
                    (query): Linear(in_features=768, out_features=768, bias=True)
                    (key): Linear(in_features=768, out_features=768, bias=True)
                    (value): Linear(in_features=768, out_features=768, bias=True)
                    (drop

In [6]:
evaluation = {}
dataloader_length = len(dataloader)

for i, batch in enumerate(dataloader):
    print(f'{i}/{dataloader_length}         \r', end='')

    outputs = model.forward(batch)

    batch_evaluation = evaluation_service.evaluate_batch(
        outputs,
        batch,
        EvaluationType.NamedEntityRecognitionMatch,
        i)

    update_dictionaries(evaluation, batch_evaluation)

228/238229/238230/238231/238232/238233/238234/238235/238236/238237/238

In [7]:
output_path = evaluation_service.save_results(evaluation)

'..\\results\\named-entity-recognition\\rnn-simple\\french\\output-None.tsv'

In [12]:
output_path = '..\\results\\named-entity-recognition\\rnn-simple\\french\\output-french-all--ft-bert-pretr-ce16-ch32-h256-e256-l1-bi-d0.80.0001-nonew-spl-ms.tsv'

In [53]:
p = subprocess.Popen([
    "python", 
    'D:\\OneDrive\\Learning\\University\\Masters-UvA\\Thesis\\challenges\\clef\\scorer\\CLEF-HIPE-2020-scorer\\clef_evaluation.py',
    '--ref', 
    'D:\\OneDrive\\Learning\\University\\Masters-UvA\\Thesis\\code\\eval-historical-texts\\data\\named-entity-recognition\\rnn-simple\\french\\HIPE-data-v1.3-test-fr.tsv',
    '--pred', f'D:\\OneDrive\\Learning\\University\\Masters-UvA\\Thesis\\code\\eval-historical-texts\\notebooks\\{output_path}',
    '--task', 'nerc_coarse',
    '--skip_check'
], stdin=subprocess.PIPE, stdout=subprocess.PIPE, stderr=subprocess.PIPE)

output, err = p.communicate()
print(output.decode('utf-8'))
error_str = err.decode('utf-8')
print(error_str)
assert p.returncode == 0

2020-07-11 04:28:37 root INFO: Datasets imported (Gold/Predictions).
2020-07-11 04:28:37 root INFO: Number of docs: 43	43
2020-07-11 04:28:37 root INFO: Number of lines: 4376	4376
2020-07-11 04:28:37 root INFO: Number of tokens: 40854	40854
2020-07-11 04:28:37 root INFO: Evaluating on ['NE-COARSE-METO'] for the following tags: {'ORG', 'TIME'}
2020-07-11 04:28:37 root INFO: Evaluating on ['NE-COARSE-LIT'] for the following tags: {'ORG', 'LOC', 'TIME', 'PROD', 'PERS'}



In [54]:
p = subprocess.Popen([
    "python", 
    'D:\\OneDrive\\Learning\\University\\Masters-UvA\\Thesis\\challenges\\clef\\scorer\\CLEF-HIPE-2020-scorer\\clef_evaluation.py',
    '--ref', 
    'D:\\OneDrive\\Learning\\University\\Masters-UvA\\Thesis\\code\\eval-historical-texts\\data\\named-entity-recognition\\rnn-simple\\french\\HIPE-data-v1.3-test-fr.tsv',
    '--pred', f'D:\\OneDrive\\Learning\\University\\Masters-UvA\\Thesis\\code\\eval-historical-texts\\notebooks\\{output_path}',
    '--task', 'nerc_fine',
    '--skip_check'
], stdin=subprocess.PIPE, stdout=subprocess.PIPE, stderr=subprocess.PIPE)

output, err = p.communicate()
print(output.decode('utf-8'))
error_str = err.decode('utf-8')
print(error_str)
assert p.returncode == 0


2020-07-11 04:29:09 root INFO: Datasets imported (Gold/Predictions).
2020-07-11 04:29:09 root INFO: Number of docs: 43	43
2020-07-11 04:29:09 root INFO: Number of lines: 4376	4376
2020-07-11 04:29:09 root INFO: Number of tokens: 40854	40854
2020-07-11 04:29:09 root INFO: Evaluating on ['NE-FINE-LIT'] for the following tags: {'ORG.ENT.PRESSAGENCY', 'LOC.ADM.NAT', 'ORG.ADM', 'LOC.ADM.SUP', 'PROD.DOCTR', 'LOC.UNK', 'LOC.FAC', 'LOC.PHYS.GEO', 'LOC.PHYS.HYDRO', 'PERS.COLL', 'LOC.ORO', 'PERS.IND', 'TIME.DATE.ABS', 'PROD.MEDIA', 'LOC.ADM.TOWN', 'ORG.ENT', 'LOC.ADM.REG', 'PERS.IND.ARTICLEAUTHOR'}
2020-07-11 04:29:09 root INFO: Evaluating on ['NE-FINE-METO'] for the following tags: {'ORG.ADM', 'TIME.DATE.ABS', 'ORG.ENT'}
2020-07-11 04:29:09 root INFO: Evaluating on ['NE-NESTED'] for the following tags: {'PROD.MEDIA', 'PERS.IND', 'LOC.ADM.TOWN', 'LOC.ADM.NAT', 'ORG.ENT', 'ORG.ADM', 'LOC.ADM.REG', 'LOC.PHYS.GEO', 'LOC.PHYS.HYDRO'}
2020-07-11 04:29:10 root INFO: Evaluating on ['NE-FINE-COMP'] for

In [63]:
coarse_file_path = 'D:\\OneDrive\\Learning\\University\\Masters-UvA\\Thesis\\code\\eval-historical-texts\\results\\named-entity-recognition\\rnn-simple\\french\\results_nerc_coarse_LANG_all.json'

fine_file_path = 'D:\\OneDrive\\Learning\\University\\Masters-UvA\\Thesis\\code\\eval-historical-texts\\results\\named-entity-recognition\\rnn-simple\\french\\results_nerc_fine_LANG_all.json'

def get_test_scores(filepath):
    with open(filepath, 'r') as json_file:
        data = json.load(json_file)
        partial_results = { tag_type : tag_values['ALL']['partial'] for tag_type, tag_values in data.items() }
        
        test_partial_f1_micro = {
            tag_type: tag_values['F1_micro'] for tag_type, tag_values in partial_results.items()
        }

        return test_partial_f1_micro

print(get_test_scores(coarse_file_path))
print(get_test_scores(fine_file_path))

{'NE-COARSE-METO': 0, 'NE-COARSE-LIT': 0.8100201748486886}
{'NE-FINE-LIT': 0.7828473413379073, 'NE-FINE-METO': 0, 'NE-NESTED': 0, 'NE-FINE-COMP': 0.5974789915966386}


In [ ]:
# Log to WANDB